<a href="https://colab.research.google.com/github/zhaomargot/energyloadforecasts/blob/main/zhaomargot_dso4_goal3_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install pycaret
!pip install pycaret
# import libraries
import pandas as pd
# from pycaret.time_series import *
# from pycaret.internal.pycaret_experiment import TimeSeriesExperiment
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from statsmodels.tsa.seasonal import STL
from statsmodels.graphics.tsaplots import plot_acf

# resolution
mpl.rcParams['figure.dpi'] = 300
plt.style.use('seaborn-whitegrid')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 463.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.8/105.8 kB 13.8 MB/s eta 0:00:00


<ipython-input-1-6ecbb4099df1>:16: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [ ]:
# import data as pandas file
ogdf = pd.read_excel("https://github.com/robertasgabrys/DSO424SPRING2023/blob/main/CompetitionData.xlsx?raw=true")

In [ ]:
# create timestamps
ogdf["Time"] = pd.date_range(start = pd.Timestamp('2002-01-01 00:00'), end = pd.Timestamp('2006-12-31 23:59'),freq='H')

# Create a column "Day" with just the date and not time
ogdf["Day"] = ogdf["Time"].dt.date

print(ogdf.head())

   Tavg  Tmed  Tmax  Tmin       Load                Time         Day
0    43  43.0  60.0  31.0  1384494.0 2002-01-01 00:00:00  2002-01-01
1    42  42.0  58.0  29.0  1392822.0 2002-01-01 01:00:00  2002-01-01
2    41  41.0  57.0  31.0  1407887.0 2002-01-01 02:00:00  2002-01-01
3    41  41.0  56.0  30.0  1438658.0 2002-01-01 03:00:00  2002-01-01
4    40  41.0  53.0  29.0  1484046.0 2002-01-01 04:00:00  2002-01-01


In [ ]:

df = ogdf.groupby("Day").agg({"Tavg": "mean", "Tmed": "mean", "Tmax": "mean", "Tmin": "mean", "Load": "max"})[["Tavg", "Tmed", "Tmax", "Tmin", "Load"]].reset_index()
print(df)

# Find the daily peak loads
#peakseries = df.groupby(["Day"])["Load"].max()
# Turn into a dataframe
peakdf = pd.DataFrame({"Day": peakseries.index, "Load": peakseries.values})

# Use an inner join to preserve Tavg / Tmed / Tmax / Tmin / Hour data for the daily peak
df = peakdf.merge(df, how = 'inner', on = ["Day", "Load"])
df["Time"] = ogdf["Time"]

# Get hour as a number

#df["Hour"] = df["Time"].dt.strftime("%H")
#df["Hour"] = df["Hour"].astype(int)
#print(df)
#peakdf["Time"] = ogdf["Time"]
#peakdf["Hour"] = peakdf["Time"].dt.strftime("%H")
#peakdf["Hour"] = peakdf["Hour"].astype(int)
#print(peakdf)

# append hour to DF
#df["Hour"] = peakdf["Hour"]
#df["Time"] = peakdf["Time"]

print(df)

             Day       Tavg       Tmed       Tmax       Tmin       Load
0     2002-01-01  48.500000  49.125000  61.125000  36.166667  1871982.0
1     2002-01-02  51.291667  54.916667  64.041667  36.791667  1627644.0
2     2002-01-03  43.625000  42.791667  68.708333  34.541667  2260145.0
3     2002-01-04  37.375000  37.666667  46.416667  30.333333  2545548.0
4     2002-01-05  45.500000  46.083333  64.458333  31.750000  2588347.0
...          ...        ...        ...        ...        ...        ...
1821  2006-12-27  47.291667  47.375000  58.750000  40.041667        NaN
1822  2006-12-28  57.041667  58.291667  70.375000  46.791667        NaN
1823  2006-12-29  65.541667  65.583333  75.125000  58.541667        NaN
1824  2006-12-30  69.166667  69.708333  76.500000  61.875000        NaN
1825  2006-12-31  71.541667  72.208333  77.083333  64.333333        NaN

[1826 rows x 6 columns]


NameError: ignored

In [ ]:
df["Month"] = [i.month for i in df["Day"]]
df["Year"] = [i.year for i in df["Day"]]
df["Day_of_week"] = [i.isoweekday() for i in df["Day"]]
df["Day_of_year"] = [pd.Period(i, freq='D').day_of_year for i in df["Day"]]

df.set_index("Day", inplace=True)
df.index =  pd.to_datetime(df.index, format='%Y-%m-%d')

print(df)

# for training models, create df4 with only data from 2002 to 2005
df4 = df.loc[df["Time"].dt.year < 2006]

# Create Hour column (integer) from Time datetime column
# Shows hour of daily peak
df4["Hour"] = df4["Time"].dt.strftime("%H")
df4["Hour"] = df4["Hour"].astype(int)

# drop time & load
df4.drop(columns=["Time", "Load"], inplace=True)

print(df4)

In [ ]:
df4.reset_index(drop=True, inplace=True)

In [ ]:
 # train size (2002-2004) = 1096 days out of 1461 = 0.750171116,
 from pycaret.regression import *
 s = setup(df4, target = "Hour", train_size=0.750171116,
              data_split_shuffle = False, fold_strategy = 'timeseries', fold = 3,
              numeric_features = ['Day_of_year', "Tmax", "Tmin", "Year", "Tavg", "Tmed",],
              categorical_features = ['Month', 'Day_of_week'], n_features_to_select=2, transformation=True, verbose = False, use_gpu=True, session_id = 123)

# silent = True

In [ ]:
best_model = compare_models(sort = 'MAE', verbose=False)

In [ ]:
all_results = []
p = pull().iloc[0:1]
#p['time_series'] = str(i)
all_results.append(p)
# Training (2002-2004) MAE
print(p)
# MAE = 1.5583 w all columns

In [ ]:
best = create_model("rf")

In [ ]:
# Test (2005) MAE
prediction_holdout = predict_model(best)
# MAE = 1.6745

In [ ]:
f = finalize_model(best)

In [ ]:
df.drop(columns=["Time", "Load"], inplace=True)
df.reset_index(drop=True, inplace=True)
df2006 = df.drop(df4.index)
df2006.reset_index(drop=True, inplace=True)
print(df2006)

In [ ]:
# 2006 Forecast
# use df06 which has data of 2006
predict_model(best, df2006, verbose=False)
